In [1]:
import xlwings as xw
import json
import numpy as np
import pandas as pd

import convert_to_input_json as cij

# 目標とするUA、ηAを再現する自立モデルの外皮仕様を計算し、input.jsonを作成する

In [2]:
# 暖房と冷房
ac_mode_s = np.array(['C', 'H'])

# UA値は0.2～0.87まで0.2刻み
UA_s = np.arange(start=0.4, stop=0.95, step=0.1, dtype=float)
# ηAは5%～15%まで2%刻み
eta_ac_s = np.arange(start=0.2, stop=5.0, step=0.2, dtype=float) / 100.0
# 地域区分は1～8地域
region_s = np.arange(start=6, stop=7, step=1, dtype=float)

# UA_s = np.array([0.86])
# eta_ac_s = np.array([1.98]) / 100.0
# ac_mode_s = np.array(['C'])

# 暖房、冷房の運転モード
operation_mode_s = np.array(['全館連続', '居室連続', '居室間歇'])
operation_mode_s = np.array(['全館連続', '居室間歇'])

# 蓄熱の利用（1：なし、2：あり）
TS_s = np.array([1], dtype=float)

# 全組み合わせの作成
ac_mode_array, region_array, operation_mode_array, TS_array, UA_s_array, eta_ac_s_array \
    = np.meshgrid(ac_mode_s, region_s, operation_mode_s, TS_s, UA_s, eta_ac_s)

ac_mode_array = ac_mode_array.flatten()
region_array = region_array.flatten()
operation_mode_array = operation_mode_array.flatten()
TS_array = TS_array.flatten()
UA_s_array = UA_s_array.flatten()
eta_ac_s_array = eta_ac_s_array.flatten()

In [3]:
def make_json(case: int, ac_mode: str, region: int, operation_mode: str, TS: int, ua_value: float, eta_a_value: float) ->  (bool, bool):

    print(case, ac_mode, region, operation_mode, TS, ua_value, eta_a_value)

    # Excelが画面に表示されないようにする
    xw.App(visible=False)

    # Excelファイルを開く
    excel_file_name = 'continuous_calc_input_excel_UA_etaA_step3.xlsx'
    wb = xw.Book(excel_file_name)
    sheet = wb.sheets['計算条件']
    # A3セルに暖冷房モードを入力する
    sheet.range('A3').value = ac_mode
    # B3セルに地域区分を入力
    sheet.range('B3').value = region
    # C3セルに運転モードを入力
    sheet.range('C3').value = operation_mode
    # E3セルに熱容量を入力
    sheet.range('E3').value = TS
    # I3セルに目標UA値を入力
    sheet.range('I3').value = ua_value
    # J3セルに目標ηA値を入力
    sheet.range('J3').value = eta_a_value

    # 目標UA、ηA値が担保できているかチェック
    is_ua_value_check = sheet.range('K3').value
    is_etaa_value_check = sheet.range('L3').value

    # 保存する
    wb.save()
    # 断熱性能
    insulation = f"{int(ua_value * 100.0):03d}"
    # 日射遮へい性能
    shading = f"{int(eta_a_value * 1000.0):03d}"
    # 閉じる
    wb.close()

    # エクセルからのDict書き出し
    input_dict = cij.convert_excel_to_json(excel_file=excel_file_name)

    # common
    common = {"common": {"ac_method": "air_temperature",
                            "ac_config": [{"mode": 1, "lower": 20, "upper": 27},
                                        {"mode": 2, "lower": 0, "upper": 28}]}}

    # mechanical_ventilation
    # 等級3以上および等級1、2の夏期（緑本付図3.1.8、付表 3.1.16）
    mechanical_ventilation_base = {
    "mechanical_ventilations": [
    { "id": 0, "root_type": "type3", "volume": 60.0, "root": [0, 2]},
    { "id": 1, "root_type": "type3", "volume": 60.0, "root": [1, 2]},
    { "id": 2, "root_type": "type3", "volume": 20.0, "root": [2]},
    { "id": 3, "root_type": "type3", "volume": 20.0, "root": [2, 1, 2]},
    { "id": 6, "root_type": "type3", "volume": 234.76635, "root": [4]},
    { "id": 7, "root_type": "type3", "volume": 55.89675, "root": [5]},
    { "id": 8, "root_type": "type3", "volume": 14.9058, "root": [6]},
    { "id": 9, "root_type": "type3", "volume": 161.125, "root": [7]}
    ]
    }
    
    # 隙間風はないとして計算する
    # # 等級1、2の冬期（緑本付図3.1.10、付表 3.1.16）
    # mechanical_ventilation_large_inf = {
    # "mechanical_ventilations": [
    # { "id": 0, "root_type": "type3", "volume": 72.0, "root": [2, 1, 6, 11, 13]},
    # { "id": 1, "root_type": "type3", "volume": 12.0, "root": [2, 1, 6, 11, 9]},
    # { "id": 2, "root_type": "type3", "volume": 104.0, "root": [1, 6, 11]},
    # { "id": 3, "root_type": "type3", "volume": 32.0, "root": [2, 1, 6, 11, 8, 7]},
    # { "id": 4, "root_type": "type3", "volume": 44.0, "root": [0, 6, 11, 10]},
    # { "id": 5, "root_type": "type3", "volume": 40.0, "root": [0, 6, 11, 8]},
    # { "id": 6, "root_type": "type3", "volume": 10.665, "root": [0, 6, 4]},
    # { "id": 7, "root_type": "type3", "volume": 10.665, "root": [0, 6, 5]},
    # { "id": 8, "root_type": "type3", "volume": 12.0, "root": [6, 11, 8, 7]},
    # { "id": 11, "root_type": "type3", "volume": 4.0, "root": [6, 11, 8]},
    # { "id": 9, "root_type": "natural_loop", "volume": 5000.0, "root": [1, 2]},
    # { "id": 10, "root_type": "natural_loop", "volume": 273, "root": [6, 11]},
    # { "id": 11, "root_type": "type3", "volume": 234.76635, "root": [14]},
    # { "id": 12, "root_type": "type3", "volume": 55.89675, "root": [15]},
    # { "id": 13, "root_type": "type3", "volume": 14.9058, "root": [16]},
    # { "id": 14, "root_type": "type3", "volume": 161.125, "root": [17]}
    # ]
    # }

    # equipments
    equipments_dict = {
        "equipments": {
            "heating_equipments": [],
            "cooling_equipments": []
            }
            }

    # 機械換気、設備機器を追加
    input_dict.update(**common, **mechanical_ventilation_base, **equipments_dict)

    # JSONファイルの保存
    file_name = f"{case:05d}" + '_' + str(int(region)) + ac_mode + '_' + operation_mode + '_' + str(int(TS)) + '_' + insulation + '_' + shading
    with open('input_data/' + file_name + '.json', 'w', encoding='utf-8') as json_file:
        json.dump(input_dict, json_file, indent=2, ensure_ascii=False)

    return is_ua_value_check, is_etaa_value_check

In [4]:
i = 0
result = pd.DataFrame(columns=['case', 'ac_mode', 'region', 'operation_mode', 'TS', 'ua_value', 'eta_a_value', 'ua_flg', 'etaa_flg'])
for (ac_mode, region, operation_mode, TS, ua_value, eta_a_value) \
    in zip(ac_mode_array, region_array, operation_mode_array, TS_array, UA_s_array, eta_ac_s_array):

    if ac_mode == 'C':
        f_eta = 1.0
        # f_eta = 0.932 / 0.8
    else:
        f_eta = 1.0
        # f_eta = 0.8188 / 2

    if not((region == 8 and ac_mode == 'H') or (ac_mode == 'C' and operation_mode == '居室連続')):
        ua_flg, etaa_flg = make_json(
            case=i,
            ac_mode=ac_mode,
            region=region,
            operation_mode=operation_mode,
            TS=TS,
            ua_value=ua_value,
            eta_a_value=eta_a_value * f_eta)
        result.at[i, 'case'] = i
        result.at[i, 'ac_mode'] = ac_mode
        result.at[i, 'region'] = region
        result.at[i, 'operation_mode'] = operation_mode
        result.at[i, 'TS'] = TS
        result.at[i, 'ua_value'] = ua_value
        result.at[i, 'eta_a_value'] = eta_a_value
        result.at[i, 'ua_flg'] = ua_flg
        result.at[i, 'etaa_flg'] = etaa_flg
        i += 1

result.to_csv('result.csv')

0 C 6.0 全館連続 1.0 0.4 0.002
1 C 6.0 全館連続 1.0 0.4 0.004
2 C 6.0 全館連続 1.0 0.4 0.006000000000000001
3 C 6.0 全館連続 1.0 0.4 0.008
4 C 6.0 全館連続 1.0 0.4 0.01
5 C 6.0 全館連続 1.0 0.4 0.012
6 C 6.0 全館連続 1.0 0.4 0.014000000000000002
7 C 6.0 全館連続 1.0 0.4 0.016
8 C 6.0 全館連続 1.0 0.4 0.018000000000000002
9 C 6.0 全館連続 1.0 0.4 0.02
10 C 6.0 全館連続 1.0 0.4 0.022000000000000002
11 C 6.0 全館連続 1.0 0.4 0.024000000000000004
12 C 6.0 全館連続 1.0 0.4 0.026000000000000006
13 C 6.0 全館連続 1.0 0.4 0.028000000000000004
14 C 6.0 全館連続 1.0 0.4 0.030000000000000006
15 C 6.0 全館連続 1.0 0.4 0.032
16 C 6.0 全館連続 1.0 0.4 0.034
17 C 6.0 全館連続 1.0 0.4 0.036000000000000004
18 C 6.0 全館連続 1.0 0.4 0.038000000000000006
19 C 6.0 全館連続 1.0 0.4 0.04
20 C 6.0 全館連続 1.0 0.4 0.042
21 C 6.0 全館連続 1.0 0.4 0.044000000000000004
22 C 6.0 全館連続 1.0 0.4 0.046000000000000006
23 C 6.0 全館連続 1.0 0.4 0.04800000000000001
24 C 6.0 全館連続 1.0 0.5 0.002
25 C 6.0 全館連続 1.0 0.5 0.004
26 C 6.0 全館連続 1.0 0.5 0.006000000000000001
27 C 6.0 全館連続 1.0 0.5 0.008
28 C 6.0 全館連続 1.0 0.